<a href="https://colab.research.google.com/github/once-returner/address-development/blob/master/openstreetmap_address_development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Address Development without a Normalized Jurisdiction Database - SJ Porter

## Introduction

This proof of concept uses a search string to find an address and surrounding counties based on a provided radius (in kilometers).

## Resources

- [Open Street Maps API for Python (OsmApi) Documentation](http://osmapi.metaodi.ch/)
- [OsmApi on PyPi](https://pypi.org/project/osmapi/)
- [Nominatim - Query OpenStreetMap data based on name and address](https://github.com/openstreetmap/Nominatim)
- [GeoPy - Get Lat/Long and full address from an address](https://geopy.readthedocs.io/en/latest/)
- [GeoPy on PyPi](https://pypi.org/project/geopy/)

## Define the Search Parameters

Here is where we will define our free-text address and the number of kilometers we want to develop surrounding the address.

In [0]:
# search criteria
search_string = "917 Chapin Rd, Chapin, SC"
search_kilometers = 20

## Install Dependencies

In order to return results, we must install the various utilities required to perform an address search.

In [0]:
# first, we must install non-standard dependencies

!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

## Locate the Address Being Searched

Here is where we find the address that the user is searching for.

In [0]:
geolocator = Nominatim(user_agent="address_development")
location = geolocator.geocode(search_string)

print(f'Address Match: {location.address}')
print(f'Coordinates: {(location.latitude, location.longitude)}')

Address Match: 917, Chapin Road, Chapin, Lexington County, South Carolina, 29036, United States of America
Coordinates: (34.1623905555556, -81.3452802222222)


## Define a Map Querying Function

This function will help us repeatedly read from OpenStreetMap with minimal coding.

In [0]:
def map_query(longitude:float, latitude:float) -> dict:
  '''Query OpenStreetMap for a long/lat coordinate and return relevant metadata.'''
  url = f'https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={latitude}&lon={longitude}&zoom=18&addressdetails=1&namedetails=1'
  data = json.loads(requests.get(url).text)
  return data

## Do Some Math :'(

Since we will need to develop counties surrounding our provided address, we will create a function to return a set of coordinates in a radius around a given point.

A special thank you goes out to [Mike T.](https://gis.stackexchange.com/users/1872/mike-t) on [Stack Exchange](https://gis.stackexchange.com/questions/289044/creating-buffer-circle-x-kilometers-from-point-using-python) for providing the function logic.

In [0]:
# instantiate a geographic projector
proj = pyproj.Proj(init='epsg:4326')

# define a function that will return a set of coordinates in N radius (km) from a point
def geodesic_point_buffer(lat, lon, km):
    # azimuthal equidistant projection
    project = partial(
        pyproj.transform,
        pyproj.Proj(f'+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'),
        proj)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

## Query the Address Metadata

Now that we have longitude/latitude coordinates for our address (or whatever returned from the search), we can query regional boundaries via OpenStreetMap.

In [0]:
# query the main location data
location_data = map_query(longitude=location.longitude, latitude=location.latitude)

# print the location data
print(f'Address:       {location.address}')
print(f'County:        {location_data["address"]["county"]}')
print(f'State:         {location_data["address"]["state"]}')
print(f'Zip Code:      {location_data["address"]["postcode"]}')
print(f'Country:       {location_data["address"]["country"]}')
print(f'Country Code:  {location_data["address"]["country_code"]}\n\n')

# optionally, we could print all of the data being returned
# pprint.pprint(location_data)

Address:       917, Chapin Road, Chapin, Lexington County, South Carolina, 29036, United States of America
County:        Lexington County
State:         South Carolina
Zip Code:      29036
Country:       United States of America
Country Code:  us




## Query the Surrounding Area

Here is where we will query the surrounding area (defined above). The search is performed as a radius around the original coordinates in kilometers.

In [0]:
# query the surrounding area
surrounding_area_data = {}
surrounding_area_counter = 0

for bounding_point in geodesic_point_buffer(location.latitude, location.longitude, search_kilometers):
  surrounding_area_data.update({ str(surrounding_area_counter): map_query(longitude=bounding_point[0], latitude=bounding_point[1]) })
  surrounding_area_counter += 1

print(f'Total query points for the surrounding area: {surrounding_area_counter + 1}')

Total query points for the surrounding area: 67


## Determine Count(ies) within the Search Parameters
Now that we have location data for the address being searched as well as the surrounding area, we can easily determine which count(ies) fall within the radius.

In [0]:
# determine counties to develop
counties_to_develop = pd.DataFrame(columns=['State','County'])
counties_counter = 1

counties_to_develop = counties_to_develop.append({ 'State': location_data["address"]["state"], 'County': location_data["address"]["county"] }, ignore_index=True)

for key, value in surrounding_area_data.items():
  counties_counter += 1
  if value["category"] == 'place':
    counties_to_develop = counties_to_develop.append({ 'State': value["address"]["state"], 'County': value["address"]["county"] }, ignore_index=True)

print(f'Counties within {search_kilometers} kilometers of "{location.address}":\n')
print(counties_to_develop[['State', 'County']].drop_duplicates())

Counties within 20 kilometers of "917, Chapin Road, Chapin, Lexington County, South Carolina, 29036, United States of America":

             State            County
0   South Carolina  Lexington County
10  South Carolina     Saluda County
14  South Carolina   Newberry County
20  South Carolina  Fairfield County
